In [32]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

import sys

# Append the path to Pydoop to sys.path
#sys.path.append("/usr/local/lib/python3.8/dist-packages")

Spark Configuration

In [33]:
# Spark Session - 2 CORE - imporvement CONFIGURATION!!
# spark_session = SparkSession.builder\
#     .master("local[2]")\
#     .appName("pseudo_spark_nora")\
#     .getOrCreate()

# Set Hadoop configuration directory
#os.environ['HADOOP_CONF_DIR'] = '/path/to/hadoop/conf'
#spark_session.stop()
#os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

spark_session = SparkSession.builder \
    .appName("HDFS_Connection_Test_nora") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# num executors 
# executor memory RAM 


# spark_session = SparkSession.builder \
#     .appName("HDFS_Connection_Test_nora") \
#     .master("yarn") \
#     .config("spark.executor.instances", "2") \
#     .config("spark.hadoop.fs.default.name", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.defaultFS", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
#     .config("spark.hadoop.dfs.server.namenode.class", "org.apache.hadoop.hdfs.server.namenode.NameNode") \
#     .config("spark.hadoop.conf", "org.apache.hadoop.hdfs.HdfsConfiguration") \
#     .getOrCreate()

#.config("spark.dynamicAllocation.enabled", False)\
#.config("spark.cores.max", 4)\


#.config("spark.jars.packages", "LLNL:spark-hdf5:0.0.4") \
# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

File Path

In [36]:
# # Use Spark to list files in HDFS directory
# hdfs_files = spark_session._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
#     .listStatus(spark_session._jvm.org.apache.hadoop.fs.Path(directory_path))
#path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset_ASCI_analysis_songs/TRAAAAW128F429D538_songs.asci'
path = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI/'

In [37]:
file_paths = spark_context.parallelize(path)
file_paths

ParallelCollectionRDD[1175] at readRDDFromFile at PythonRDD.scala:289

In [38]:
file_first = file_paths.first()
file_first

24/03/14 23:29:46 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/14 23:29:46 INFO DAGScheduler: Got job 387 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/14 23:29:46 INFO DAGScheduler: Final stage: ResultStage 395 (runJob at PythonRDD.scala:181)
24/03/14 23:29:46 INFO DAGScheduler: Parents of final stage: List()
24/03/14 23:29:46 INFO DAGScheduler: Missing parents: List()
24/03/14 23:29:46 INFO DAGScheduler: Submitting ResultStage 395 (PythonRDD[1176] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/14 23:29:46 INFO MemoryStore: Block broadcast_750 stored as values in memory (estimated size 5.9 KiB, free 352.4 MiB)
24/03/14 23:29:46 INFO MemoryStore: Block broadcast_750_piece0 stored as bytes in memory (estimated size 3.8 KiB, free 352.4 MiB)
24/03/14 23:29:46 INFO BlockManagerInfo: Added broadcast_750_piece0 in memory on master-node:44443 (size: 3.8 KiB, free: 365.1 MiB)
24/03/14 23:29:46 INFO SparkContext: Created broadcast 750 

'h'

In [42]:
#file_paths = spark_context.wholeTextFiles(path).map(lambda x: x[0]).collect()
file_paths = spark_context.wholeTextFiles(path).map(lambda x: x[0]).take(10)

24/03/14 23:33:27 INFO MemoryStore: Block broadcast_756 stored as values in memory (estimated size 344.0 KiB, free 364.4 MiB)
24/03/14 23:33:27 INFO MemoryStore: Block broadcast_756_piece0 stored as bytes in memory (estimated size 33.3 KiB, free 364.4 MiB)
24/03/14 23:33:27 INFO BlockManagerInfo: Added broadcast_756_piece0 in memory on master-node:44443 (size: 33.3 KiB, free: 366.1 MiB)
24/03/14 23:33:27 INFO SparkContext: Created broadcast 756 from wholeTextFiles at NativeMethodAccessorImpl.java:0
24/03/14 23:33:27 INFO FileInputFormat: Total input files to process : 10000
24/03/14 23:33:28 INFO FileInputFormat: Total input files to process : 10000
24/03/14 23:33:28 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/14 23:33:28 INFO DAGScheduler: Got job 390 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/14 23:33:28 INFO DAGScheduler: Final stage: ResultStage 398 (runJob at PythonRDD.scala:181)
24/03/14 23:33:28 INFO DAGScheduler: Parents of final sta

In [43]:
all_rows = []

for file_path in file_paths:
    # Lese die Datei und bereinige den Inhalt
    rdd = spark_context.textFile(file_path)
    content = rdd.map(lambda line: line.replace('\n', '').replace(' ', '').replace(',', ''))
    
    # Erstelle Zeilen für jede Datei
    rows = [Row(column1=data[5], column2=data[6], column3=data[28], column4=data[25], column5=data[23], column6=data[29], column7=data[30], column8=data[31]) for data in [content]]
    
    # Füge die Zeilen zur gemeinsamen Liste hinzu
    all_rows.extend(rows.collect())

24/03/14 23:33:34 INFO MemoryStore: Block broadcast_758 stored as values in memory (estimated size 358.7 KiB, free 364.1 MiB)
24/03/14 23:33:34 INFO MemoryStore: Block broadcast_758_piece0 stored as bytes in memory (estimated size 33.1 KiB, free 364.0 MiB)
24/03/14 23:33:34 INFO BlockManagerInfo: Added broadcast_758_piece0 in memory on master-node:44443 (size: 33.1 KiB, free: 366.1 MiB)
24/03/14 23:33:34 INFO SparkContext: Created broadcast 758 from textFile at <unknown>:0


TypeError: 'PipelinedRDD' object is not subscriptable

24/03/15 00:02:40 INFO BlockManagerInfo: Removed broadcast_756_piece0 on master-node:44443 in memory (size: 33.3 KiB, free: 366.1 MiB)
24/03/15 00:02:40 INFO BlockManagerInfo: Removed broadcast_756_piece0 on worker-node-2:35017 in memory (size: 33.3 KiB, free: 366.2 MiB)
24/03/15 00:02:40 INFO BlockManagerInfo: Removed broadcast_757_piece0 on master-node:44443 in memory (size: 5.0 KiB, free: 366.1 MiB)
24/03/15 00:02:40 INFO BlockManagerInfo: Removed broadcast_757_piece0 on worker-node-2:35017 in memory (size: 5.0 KiB, free: 366.2 MiB)


In [ ]:
all_rows = []

for file_path in file_paths:
    # Lese die Datei und bereinige den Inhalt
    rdd = spark_context.textFile(file_path)
    content = rdd.map(lambda line: line.replace('\n', '').replace(' ', '').replace(',', '')).collect()
    
    # Erstelle Zeilen für jede Datei
    rows = [Row(column1=data[5], column2=data[6], column3=data[28], column4=data[25], column5=data[23], column6=data[29], column7=data[30], column8=data[31]) for data in [content]]
    
    # Füge die Zeilen zur gemeinsamen Liste hinzu
    all_rows.extend(rows)

In [ ]:
# Erstelle DataFrame aus den gesammelten Zeilen
schema = sqlContext.createDataFrame(all_rows)

# Register the DataFrame as a table
schema.registerTempTable("combined_data_table")

# Zeige das Ergebnis
result = sqlContext.sql("SELECT * FROM combined_data_table")
result.show()

In [ ]:
schema = StructType([
    StructField("duration", True),
    StructField("end_of_fade_in", FloatType(), True),
    StructField("start_of_fade_out", FloatType(), True),
    StructField("loudness", FloatType(), True),
    StructField("key", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("time_signature_confidence", FloatType(), True),
])

In [ ]:
# Erstelle DataFrame aus den gesammelten Zeilen
df = sqlContext.createDataFrame(all_rows, schema)


In [ ]:
print("Number of rows in the DataFrame:", df.count())

In [ ]:
df.show()

In [ ]:
# Führe die angegebene Abfrage aus, um durchschnittliche Werte zu berechnen
average_values = df.agg({'column1': 'avg', 'column2': 'avg', 'column3': 'avg', 'column4': 'avg', 'column5': 'avg', 'column6': 'avg', 'column7': 'avg', 'column8': 'avg'}).collect()[0]
print("Durchschnittswerte:")
for col_name, avg_value in average_values.asDict().items():
    print(f"{col_name}: {avg_value}")

In [ ]:
rdd = spark_context.textFile('hdfs://master-node:9000/user/hadoop/MillionSongSubset_ASCI_analysis_songs/TRAAAAW128F429D538_songs.asci')

In [ ]:
rdd = rdd.map(lambda line: line.strip('{}').split(','))

In [ ]:
rdd = rdd.map(lambda row: [str(elem) for elem in row])

In [ ]:
df = rdd.toDF()

In [ ]:
df.show()

# 3) H5Py file object file creation from the H5 file 

# Debug: Only one h5File

In [ ]:
h5_file_paths = ['hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5']
h5_file_path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5'

# Version 1

In [ ]:
def extract_info(file_path):
    with h5py.File(file_path, 'r') as file:
        song = file['analysis']['songs']
        np_song = np.array(song[0])
        selected_features = np_song[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
        return selected_features.tolist()
        #print(f"File found: {file_path}")
         #return file_path
    # except FileNotFoundError:
    #     print(f"File not found: {file_path}")
    #     return None

In [ ]:
file_paths = spark_context.parallelize(h5_file_path)
files_infos_rdd = file_paths.map(extract_info)

In [ ]:
print(files_infos_rdd.collect())

# 4) Create Spark Data Frame

In [ ]:
# Definiere das Schema für den DataFrame
schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("duration", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("loudness", FloatType(), True),
])

In [ ]:
spark_df = sqlContext.createDataFrame(files_infos_rdd.flatMap(lambda x: x), schema=schema)

In [ ]:
# Zeige den DataFrame-Inhalt
spark_df.show()

# Calculation (Average)

In [ ]:
average_values = spark_df.agg({'danceability': 'avg', 'duration': 'avg', 'tempo': 'avg', 'energy': 'avg', 'loudness': 'avg'}).collect()[0]
print("Average Values::")
for col_name, avg_value in average_values.asDict().items():
    print(f"{col_name}: {avg_value}")

# STOP SPARK

In [ ]:
# release the cores
spark_context.stop()